<a href="https://colab.research.google.com/github/AsifRifat/Cricket-Outcome-Predictor/blob/main/CricketWinProbabilityPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import necessary libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [55]:
import os
import pandas as pd
import numpy as np
import glob
import pickle
from pandas import DataFrame
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# **Import ODI INFO Dataset**

In [3]:
%cd /content/drive/My\ Drive/cricket data/odi_info

/content/drive/My Drive/cricket data/odi_info


In [ ]:
pd.read_csv("64857_info.csv", error_bad_lines=False)

b'Skipping line 21: expected 3 fields, saw 4\nSkipping line 22: expected 3 fields, saw 4\nSkipping line 23: expected 3 fields, saw 4\nSkipping line 24: expected 3 fields, saw 4\nSkipping line 25: expected 3 fields, saw 4\nSkipping line 26: expected 3 fields, saw 4\nSkipping line 27: expected 3 fields, saw 4\nSkipping line 28: expected 3 fields, saw 4\nSkipping line 29: expected 3 fields, saw 4\nSkipping line 30: expected 3 fields, saw 4\nSkipping line 31: expected 3 fields, saw 4\nSkipping line 32: expected 3 fields, saw 4\nSkipping line 33: expected 3 fields, saw 4\nSkipping line 34: expected 3 fields, saw 4\nSkipping line 35: expected 3 fields, saw 4\nSkipping line 36: expected 3 fields, saw 4\nSkipping line 37: expected 3 fields, saw 4\nSkipping line 38: expected 3 fields, saw 4\nSkipping line 39: expected 3 fields, saw 4\nSkipping line 40: expected 3 fields, saw 4\nSkipping line 41: expected 3 fields, saw 4\nSkipping line 42: expected 3 fields, saw 4\nSkipping line 43: expected 3 f

,version,2.1.0
info,balls_per_over,6
info,team,Pakistan
info,team,New Zealand
info,gender,male
info,season,2003/04
info,date,2004/01/03
info,event,Pakistan tour of New Zealand
info,match_number,1
info,venue,Eden Park
info,city,Auckland


# **Dataset Preprocessing**

In [ ]:
for f in glob.glob(r'/content/drive/My Drive/cricket data/odi_info/*.csv'):
    print(f)  # file name
    df = pd.read_csv(f, error_bad_lines=False)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(f, index=False)

In [ ]:
pd.read_csv("64857_info.csv", error_bad_lines=False)

,version,2.1.0
0,balls_per_over,6
1,team,Pakistan
2,team,New Zealand
3,gender,male
4,season,2003/04
5,date,2004/01/03
6,event,Pakistan tour of New Zealand
7,match_number,1
8,venue,Eden Park
9,city,Auckland


In [ ]:
for f in glob.glob(r'/content/drive/My Drive/cricket data/odi_info/*.csv'):
    print(f)  # file name
    df = pd.read_csv(f)
    df.version.replace(['outcome'],['winner'], inplace=True) #replace outcome by winner
    df = df.T #transpose rows into column
    df, df.columns = df[1:] , df.iloc[0] #replace 1st row as header
    df['match_id']=os.path.basename(f)
    df['match_id'] = df['match_id'].str.extract('(\d+)').astype(int) #add match_id to each file as column from filename
    #if 'winner_runs' and 'winner_wickets' not in df:
        #df['winner_runs']= np.nan
        #df['winner_wickets']= np.nan
    
    if 'winner_runs' not in df:
        df['winner_runs']=0
    if 'winner_wickets' not in df:
        df['winner_wickets']=0
    
    #if 'player_of_match' not in df:
        #df['player_of_match']=np.nan
    #cols = ['match_id','city','toss_winner', 'toss_decision','player_of_match','winner', 'winner_runs', 'winner_wickets']
    #cols = ['B','C','D','E']
    #df.assign(**{col : 0 for col in np.setdiff1d(cols,df.columns.values)})
    df = df[['match_id','city','toss_winner', 'toss_decision','winner', 'winner_runs', 'winner_wickets']]
    df.reset_index(drop=True, inplace=True)
    df.to_csv(f, index=False)

In [ ]:
pd.read_csv("64857_info.csv", error_bad_lines=False)

,match_id,city,toss_winner,toss_decision,winner,winner_runs,winner_wickets
0,64857,Auckland,New Zealand,field,New Zealand,0,4


# **Merge Preprocessed CSV Files**

In [ ]:
files = glob.glob(r'/content/drive/My Drive/cricket data/odi_info/*.csv')
combined_info = pd.concat([pd.read_csv(f, index_col=False) for f in files])

In [ ]:
combined_info.to_csv( "odi_info.csv", index=False)

In [ ]:
combined_info

,match_id,city,toss_winner,toss_decision,winner,winner_runs,winner_wickets
0,522249,Bulawayo,Bangladesh,field,Bangladesh,0,6
0,560925,St Kitts,West Indies,field,New Zealand,88,0
0,433588,Bangalore,Australia,bat,Australia,60,0
0,1027317,Antigua,West Indies,bat,England,0,4
0,1251952,Dublin,South Africa,bat,South Africa,70,0
...,...,...,...,...,...,...,...
0,256608,Kuala Lumpur,Australia,bat,no result,0,0
0,1133003,Harare,West Indies,bat,West Indies,60,0
0,1153255,NaN,India,field,India,0,3
0,1119496,Chennai,India,bat,India,26,0


In [4]:
info = pd.read_csv("odi_info.csv")

# **Import ODI Deliveries Data**

In [6]:
%cd /content/drive/My\ Drive/cricket data

/content/drive/My Drive/cricket data


In [7]:
deliveries = pd.read_csv("odi.csv")

In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
deliveries.head(5)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,0.1,Australia,Pakistan,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,0.2,Australia,Pakistan,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,0.3,Australia,Pakistan,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,0.4,Australia,Pakistan,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,0.5,Australia,Pakistan,DA Warner,TM Head,Mohammad Amir,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Data Pre-processing**

In [9]:
temp = pd.read_csv("odi.csv")
sum_runs = temp["runs_off_bat"] + temp["extras"]
temp["runs"] = sum_runs

target = temp.groupby(['match_id','innings']).sum()['runs'].reset_index()
target = target[target['innings'] == 1]
target['runs'] += 1
target.columns = ['match_id','innings','target']

target.head()

,match_id,innings,target
0,64857,1,230
2,64862,1,264
4,64864,1,148
6,64865,1,298
8,64870,1,263


In [10]:
#Keep only second innings of each ODI match
match_data = deliveries[deliveries.innings == 2]

In [11]:
#Sort a data frame by "start_date", "match_id", "innings", "ball
match_data = match_data.sort_values(by = ["start_date", "match_id", "innings", "ball" ],axis = 0,ascending = [True, True, True, True])

In [ ]:
match_data.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
811492,64857,2003/04,2004-01-03,Eden Park,2,0.1,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811493,64857,2003/04,2004-01-03,Eden Park,2,0.2,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811494,64857,2003/04,2004-01-03,Eden Park,2,0.3,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811495,64857,2003/04,2004-01-03,Eden Park,2,0.4,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811496,64857,2003/04,2004-01-03,Eden Park,2,0.5,New Zealand,Pakistan,SP Fleming,CD Cumming,Mohammad Sami,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from datetime import datetime
match_data['start_date'] = match_data['start_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [13]:
sum_runs_off_bat_extras = match_data["runs_off_bat"] + match_data["extras"]

match_data["runs"] = sum_runs_off_bat_extras

In [14]:
match_data['current_run'] = match_data.groupby('match_id')['runs'].cumsum()

In [15]:
gr2 = match_data.groupby('match_id')
match_data["total_runs"] = gr2.current_run.transform("max")

In [16]:
match_data['wicket']=(match_data['wicket_type'].ne('retired hurt') & match_data['wicket_type'].notna()).groupby(match_data['match_id']).cumsum()

In [17]:
match_data['runs_last10'] = match_data.groupby('match_id',sort=False)['runs'].rolling(min_periods=1, window=60).sum().astype("int64").values

In [18]:
match_data['wicket_last10'] = ((match_data['wicket_type'].ne('retired hurt') & match_data['wicket_type'].notna())
                      .groupby(match_data['match_id'],sort=False)
                      .rolling(min_periods=1, window=60).sum().astype("int64")
                      .values)

In [19]:
match_data['venue_avg'] = match_data['venue'].map(match_data.drop_duplicates(['match_id', 'venue']).groupby('venue')['total_runs'].mean().astype(int))

In [ ]:
match_data.tail()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,runs,current_run,total_runs,wicket,runs_last10,wicket_last10,venue_avg
300841,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.4,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164
300842,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.5,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164
300843,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.6,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,116,116,9,25,3,164
300844,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.7,Oman,Namibia,Bilal Khan,Khawar Ali,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,116,116,9,25,3,164
300845,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,42.1,Oman,Namibia,Khawar Ali,Bilal Khan,M van Lingen,0,0,NaN,NaN,NaN,NaN,NaN,caught,Khawar Ali,NaN,NaN,0,116,116,10,25,3,164


In [20]:
match_df = info.merge(target[['match_id','target']],left_on='match_id',right_on='match_id')

match_df.head()

,match_id,city,toss_winner,toss_decision,winner,winner_runs,winner_wickets,target
0,522249,Bulawayo,Bangladesh,field,Bangladesh,0,6,200
1,560925,St Kitts,West Indies,field,New Zealand,88,0,250
2,433588,Bangalore,Australia,bat,Australia,60,0,325
3,1027317,Antigua,West Indies,bat,England,0,4,226
4,1251952,Dublin,South Africa,bat,South Africa,70,0,347


In [21]:
final_df = match_data.merge(match_df,on='match_id')
final_df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,runs,current_run,total_runs,wicket,runs_last10,wicket_last10,venue_avg,city,toss_winner,toss_decision,winner,winner_runs,winner_wickets,target
0,64857,2003/04,2004-01-03,Eden Park,2,0.1,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,230,0,0,0,214,Auckland,New Zealand,field,New Zealand,0,4,230
1,64857,2003/04,2004-01-03,Eden Park,2,0.2,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,230,0,0,0,214,Auckland,New Zealand,field,New Zealand,0,4,230
2,64857,2003/04,2004-01-03,Eden Park,2,0.3,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,230,0,1,0,214,Auckland,New Zealand,field,New Zealand,0,4,230
3,64857,2003/04,2004-01-03,Eden Park,2,0.4,New Zealand,Pakistan,CD Cumming,SP Fleming,Mohammad Sami,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,230,0,2,0,214,Auckland,New Zealand,field,New Zealand,0,4,230
4,64857,2003/04,2004-01-03,Eden Park,2,0.5,New Zealand,Pakistan,SP Fleming,CD Cumming,Mohammad Sami,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,230,0,2,0,214,Auckland,New Zealand,field,New Zealand,0,4,230


In [22]:
final_df['runs_left'] = final_df['target'] - final_df['current_run']

In [23]:
final_df['legal_ball'] = (final_df['wides'].isna() & final_df['noballs'].isna()).groupby(final_df['match_id']).cumsum()

In [24]:
final_df['balls_left'] = 300 - final_df['legal_ball']

In [25]:
final_df['wicket_left'] = 10 - final_df['wicket']

In [ ]:
final_df.tail(10)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,runs,current_run,total_runs,wicket,runs_last10,wicket_last10,venue_avg,city,toss_winner,toss_decision,winner,winner_runs,winner_wickets,target,runs_left,legal_ball,balls_left,wicket_left
480493,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,40.5,Oman,Namibia,Khawar Ali,Bilal Khan,JJ Smit,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,113,116,9,22,3,164,Dubai,Namibia,bat,Namibia,110,0,227,114,245,55,1
480494,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,40.6,Oman,Namibia,Khawar Ali,Bilal Khan,JJ Smit,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,114,116,9,23,3,164,Dubai,Namibia,bat,Namibia,110,0,227,113,246,54,1
480495,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.1,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164,Dubai,Namibia,bat,Namibia,110,0,227,113,247,53,1
480496,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.2,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164,Dubai,Namibia,bat,Namibia,110,0,227,113,248,52,1
480497,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.3,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164,Dubai,Namibia,bat,Namibia,110,0,227,113,249,51,1
480498,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.4,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164,Dubai,Namibia,bat,Namibia,110,0,227,113,250,50,1
480499,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.5,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,114,116,9,23,3,164,Dubai,Namibia,bat,Namibia,110,0,227,113,251,49,1
480500,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.6,Oman,Namibia,Khawar Ali,Bilal Khan,BM Scholtz,0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,116,116,9,25,3,164,Dubai,Namibia,bat,Namibia,110,0,227,111,251,49,1
480501,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,41.7,Oman,Namibia,Bilal Khan,Khawar Ali,BM Scholtz,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,116,116,9,25,3,164,Dubai,Namibia,bat,Namibia,110,0,227,111,252,48,1
480502,1302617,2021/22,2022-03-06,"ICC Academy, Dubai",2,42.1,Oman,Namibia,Khawar Ali,Bilal Khan,M van Lingen,0,0,NaN,NaN,NaN,NaN,NaN,caught,Khawar Ali,NaN,NaN,0,116,116,10,25,3,164,Dubai,Namibia,bat,Namibia,110,0,227,111,253,47,0


In [ ]:
final_df['legal_ball'].value_counts()[0]

170

In [ ]:
final_df['legal_ball'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [ ]:
#final_df.loc[final_df['legal_ball'] == 0]

In [26]:
final_df.drop(final_df.index[final_df['legal_ball'] == 0], inplace = True)

In [27]:
# crr = current runs/overs bowled
final_df['crr'] = final_df['current_run']/(final_df['legal_ball']/6)
final_df['crr'] = final_df['crr'].round(decimals =2)

In [ ]:
final_df['runs_left'].unique()

array([230, 229, 228, 227, 222, 221, 220, 219, 218, 216, 215, 213, 212,
       209, 205, 204, 203, 202, 200, 198, 197, 196, 195, 194, 190, 189,
       185, 181, 180, 176, 175, 174, 173, 172, 168, 166, 165, 164, 160,
       159, 158, 156, 154, 150, 148, 146, 145, 144, 143, 142, 141, 140,
       139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127,
       123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 110, 109, 108,
       107, 106, 105, 104, 103, 102, 101,  99,  95,  94,  93,  92,  91,
        89,  88,  87,  86,  85,  80,  79,  78,  77,  76,  75,  74,  73,
        72,  68,  67,  66,  65,  64,  63,  61,  60,  59,  58,  57,  56,
        52,  51,  50,  49,  47,  46,  44,  38,  37,  36,  35,  34,  33,
        32,  31,  27,  26,  22,  18,  17,  13,  12,  10,   4,   2,   0,
       289, 285, 282, 281, 277, 276, 275, 274, 273, 272, 271, 269, 265,
       264, 261, 260, 256, 255, 254, 253, 249, 245, 244, 243, 242, 240,
       238, 237, 236, 235, 224, 223, 217, 211, 208, 206, 201, 19

In [ ]:
final_df['runs_left'].unique()

array([230, 229, 228, 227, 222, 221, 220, 219, 218, 216, 215, 213, 212,
       209, 205, 204, 203, 202, 200, 198, 197, 196, 195, 194, 190, 189,
       185, 181, 180, 176, 175, 174, 173, 172, 168, 166, 165, 164, 160,
       159, 158, 156, 154, 150, 148, 146, 145, 144, 143, 142, 141, 140,
       139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127,
       123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 110, 109, 108,
       107, 106, 105, 104, 103, 102, 101,  99,  95,  94,  93,  92,  91,
        89,  88,  87,  86,  85,  80,  79,  78,  77,  76,  75,  74,  73,
        72,  68,  67,  66,  65,  64,  63,  61,  60,  59,  58,  57,  56,
        52,  51,  50,  49,  47,  46,  44,  38,  37,  36,  35,  34,  33,
        32,  31,  27,  26,  22,  18,  17,  13,  12,  10,   4,   2, 289,
       285, 282, 281, 277, 276, 275, 274, 273, 272, 271, 269, 265, 264,
       261, 260, 256, 255, 254, 253, 249, 245, 244, 243, 242, 240, 238,
       237, 236, 235, 224, 223, 217, 211, 208, 206, 201, 199, 19

In [ ]:
final_df.loc[final_df['runs_left'] == -2]

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,runs,current_run,total_runs,wicket,runs_last10,wicket_last10,venue_avg,city,toss_winner,toss_decision,winner,winner_runs,winner_wickets,target,runs_left,legal_ball,balls_left,wicket_left,crr


In [28]:
final_df = final_df.drop(final_df.index[final_df['current_run'] >= final_df['target']])

In [29]:
final_df = final_df.drop(final_df.index[final_df['match_id'].isin([668963, 247477, 247488, 300438, 433589, 656491, 915779, 1188626, 275789, 540169,
                                                                   601612, 1179273])])
#final_df.drop(final_df.index[final_df['match_id'] == 668963], inplace = True)

In [ ]:
final_df['balls_left'].value_counts()[0]

173

In [30]:
final_df.drop(final_df.index[final_df['balls_left'] == 0], inplace = True)

In [31]:
# rrr = runs left/overs left
final_df['rrr'] = final_df['runs_left']/(final_df['balls_left']/6)
final_df['rrr'] = final_df['rrr'].round(decimals =2)

In [32]:
final_df.loc[final_df['winner'] == final_df['batting_team'], 'winner'] = 1
final_df.loc[final_df['winner'] == final_df['bowling_team'], 'winner'] = 0

In [ ]:
final_df['winner'].unique()

array([1, 0, 'no result', 'tie'], dtype=object)

In [33]:
final_df = final_df[final_df.winner != 'tie']
final_df = final_df[final_df.winner != 'no result']

In [34]:
final_df = final_df[['batting_team','bowling_team','venue','toss_winner','toss_decision','ball','runs_left','balls_left',
                     'wicket_left','runs_last10','wicket_last10','crr','rrr','target','winner']]

In [35]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(final_df[['batting_team', 'bowling_team', 'venue', 'toss_winner', 'toss_decision']])
final_df[['batting_team', 'bowling_team', 'venue', 'toss_winner', 'toss_decision']] = enc.transform(final_df[['batting_team', 'bowling_team', 'venue', 'toss_winner', 'toss_decision']])

In [38]:
final_df.sample(5)

,batting_team,bowling_team,venue,toss_winner,toss_decision,ball,runs_left,balls_left,wicket_left,runs_last10,wicket_last10,crr,rrr,target,winner
428780,18.0,3.0,182.0,16.0,1.0,18.2,209,190,8,54,1,5.40,6.60,308,0
75178,5.0,9.0,131.0,4.0,1.0,12.1,372,227,8,35,1,3.45,9.83,414,0
188278,16.0,25.0,150.0,24.0,0.0,27.4,34,134,10,52,0,4.66,1.52,163,1
280789,10.0,7.0,52.0,8.0,1.0,24.3,61,153,10,69,0,5.96,2.39,207,1
279641,21.0,3.0,65.0,19.0,1.0,33.4,109,98,8,73,0,6.50,6.67,328,1


In [39]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469559 entries, 0 to 480502
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   batting_team   469559 non-null  float64
 1   bowling_team   469559 non-null  float64
 2   venue          469559 non-null  float64
 3   toss_winner    469559 non-null  float64
 4   toss_decision  469559 non-null  float64
 5   ball           469559 non-null  float64
 6   runs_left      469559 non-null  int64  
 7   balls_left     469559 non-null  int64  
 8   wicket_left    469559 non-null  int64  
 9   runs_last10    469559 non-null  int64  
 10  wicket_last10  469559 non-null  int64  
 11  crr            469559 non-null  float64
 12  rrr            469559 non-null  float64
 13  target         469559 non-null  int64  
 14  winner         469559 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 57.3+ MB


In [43]:
final_df.describe()

,batting_team,bowling_team,venue,toss_winner,toss_decision,ball,runs_left,balls_left,wicket_left,runs_last10,wicket_last10,crr,rrr,target
count,469559.000000,469559.000000,469559.000000,469559.000000,469559.000000,469559.000000,469559.000000,469559.000000,469559.000000,469559.000000,469559.00000,469559.000000,469559.000000,469559.000000
mean,14.457061,13.400910,112.993268,12.633107,0.471706,21.349505,153.361590,170.595235,7.080222,42.870811,1.23961,4.790180,6.153815,258.065834
std,7.963157,7.778119,56.937552,7.634087,0.499199,13.185784,81.377289,79.129969,2.356238,18.381213,1.16933,1.544961,8.918642,60.249018
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,1.000000,-1.000000,0.000000,0.000000,0.00000,0.000000,-1032.000000,36.000000
25%,7.000000,7.000000,65.000000,6.000000,0.000000,10.200000,90.000000,107.000000,6.000000,32.000000,0.00000,3.950000,4.130000,221.000000
50%,16.000000,15.000000,119.000000,14.000000,0.000000,20.500000,152.000000,176.000000,8.000000,44.000000,1.00000,4.710000,5.570000,258.000000
75%,22.000000,21.000000,162.000000,20.000000,1.000000,32.100000,212.000000,238.000000,9.000000,55.000000,2.00000,5.550000,7.020000,299.000000
max,26.000000,25.000000,208.000000,24.000000,1.000000,49.700000,481.000000,299.000000,10.000000,125.000000,8.00000,66.000000,1044.000000,482.000000


In [44]:
final_df.corr()

,batting_team,bowling_team,venue,toss_winner,toss_decision,ball,runs_left,balls_left,wicket_left,runs_last10,wicket_last10,crr,rrr,target
batting_team,1.000000,-0.106447,-0.042050,0.411545,0.064081,-0.004099,0.055271,0.004098,-0.027713,-0.030339,0.038935,-0.057841,0.023524,0.045365
bowling_team,-0.106447,1.000000,-0.026210,0.476315,0.052393,-0.022539,-0.048843,0.022642,0.031963,-0.027067,-0.023646,-0.024011,-0.025217,-0.105168
venue,-0.042050,-0.026210,1.000000,-0.038177,-0.059642,-0.002706,0.022281,0.002694,0.001765,0.002590,-0.003660,0.003568,0.005758,0.029447
toss_winner,0.411545,0.476315,-0.038177,1.000000,0.080868,-0.007272,0.010168,0.007307,0.006396,-0.027849,-0.000325,-0.055123,0.001071,-0.016594
toss_decision,0.064081,0.052393,-0.059642,0.080868,1.000000,-0.004709,0.006295,0.004691,-0.017953,0.000132,0.011339,-0.017739,0.006180,-0.002948
ball,-0.004099,-0.022539,-0.002706,-0.007272,-0.004709,1.000000,-0.706885,-0.999959,-0.741816,0.545149,0.274595,0.068525,0.183653,0.130806
runs_left,0.055271,-0.048843,0.022281,0.010168,0.006295,-0.706885,1.000000,0.706836,0.420110,-0.441093,-0.109407,-0.132189,0.068756,0.546446
balls_left,0.004098,0.022642,0.002694,0.007307,0.004691,-0.999959,0.706836,1.000000,0.741795,-0.545293,-0.274658,-0.068325,-0.183700,-0.130875
wicket_left,-0.027713,0.031963,0.001765,0.006396,-0.017953,-0.741816,0.420110,0.741795,1.000000,-0.271510,-0.645727,0.095176,-0.218978,-0.150852
runs_last10,-0.030339,-0.027067,0.002590,-0.027849,0.000132,0.545149,-0.441093,-0.545293,-0.271510,1.000000,0.081703,0.522312,0.059033,0.199999


In [41]:
#Shuffle data to avoid biasness
final_df = final_df.sample(final_df.shape[0])

In [42]:
final_df.dropna(inplace=True)

In [53]:
X = final_df.iloc[:,:-1]
y = final_df.iloc[:,-1]
y = y.astype('int')

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.sample(1)

,batting_team,bowling_team,venue,toss_winner,toss_decision,ball,runs_left,balls_left,wicket_left,runs_last10,wicket_last10,crr,rrr,target
199024,West Indies,India,"Sabina Park, Kingston",West Indies,field,41.3,83,51,8,51,0,4.07,9.76,252


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','venue'])
]
,remainder='passthrough')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])


In [ ]:
pipe.fit(X_train,y_train)

In [56]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)

print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

0.828477723826561
[[39432  7988]
 [ 8120 38372]]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83     47420
           1       0.83      0.83      0.83     46492

    accuracy                           0.83     93912
   macro avg       0.83      0.83      0.83     93912
weighted avg       0.83      0.83      0.83     93912



In [57]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)
pred1 = nb.predict(X_test)

print(accuracy_score(y_test, pred1))
print(confusion_matrix(y_test, pred1))
print(classification_report(y_test, pred1))

0.7640344151972059
[[29082 18338]
 [ 3822 42670]]
              precision    recall  f1-score   support

           0       0.88      0.61      0.72     47420
           1       0.70      0.92      0.79     46492

    accuracy                           0.76     93912
   macro avg       0.79      0.77      0.76     93912
weighted avg       0.79      0.76      0.76     93912



In [58]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
pred2 = sgd.predict(X_test)

print(accuracy_score(y_test, pred2))
print(confusion_matrix(y_test, pred2))
print(classification_report(y_test, pred2))

0.6874414345344578
[[18579 28841]
 [  512 45980]]
              precision    recall  f1-score   support

           0       0.97      0.39      0.56     47420
           1       0.61      0.99      0.76     46492

    accuracy                           0.69     93912
   macro avg       0.79      0.69      0.66     93912
weighted avg       0.80      0.69      0.66     93912



In [59]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
pred3 = knn.predict(X_test)

print(accuracy_score(y_test, pred3))
print(confusion_matrix(y_test, pred3))
print(classification_report(y_test, pred3))

0.998594428826987
[[47357    63]
 [   69 46423]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     47420
           1       1.00      1.00      1.00     46492

    accuracy                           1.00     93912
   macro avg       1.00      1.00      1.00     93912
weighted avg       1.00      1.00      1.00     93912



In [60]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
pred4 = dtree.predict(X_test)

print(accuracy_score(y_test, pred4))
print(confusion_matrix(y_test, pred4))
print(classification_report(y_test, pred4))

0.9935897435897436
[[47109   311]
 [  291 46201]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     47420
           1       0.99      0.99      0.99     46492

    accuracy                           0.99     93912
   macro avg       0.99      0.99      0.99     93912
weighted avg       0.99      0.99      0.99     93912



In [61]:
from sklearn.ensemble import RandomForestClassifier
rfm = RandomForestClassifier()
rfm.fit(X_train, y_train)
pred5 = rfm.predict(X_test)

print(accuracy_score(y_test, pred5))
print(confusion_matrix(y_test, pred5))
print(classification_report(y_test, pred5))

0.9993930488116535
[[47385    35]
 [   22 46470]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     47420
           1       1.00      1.00      1.00     46492

    accuracy                           1.00     93912
   macro avg       1.00      1.00      1.00     93912
weighted avg       1.00      1.00      1.00     93912



In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel="linear", C=0.025, probability=True, random_state=101)
svm.fit(X_train, y_train)
pred6 = svm.predict(X_test)

print(accuracy_score(y_test, pred6))
print(confusion_matrix(y_test, pred6))
print(classification_report(y_test, pred6))

In [ ]:
from xgboost import XGBClassifier
xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)
pred6 = xgbc.predict(X_test)

print(accuracy_score(y_test, pred6))
print(confusion_matrix(y_test, pred6))
print(classification_report(y_test, pred6))